# Demonstração — intenção `ingredientes` e integração Deepseek

Este notebook mostra como usar o chatbot (implementado em `chatbot_pizzaria.py`) para: 
- inspecionar intenções e pratos disponíveis;
- testar detecção de intenção `ingredientes`;
- executar uma consulta real à Deepseek/OpenRouter usando a variável de ambiente `DEEPSEEK_API_KEY`.

ATENÇÃO: não coloque sua chave diretamente no notebook (nem faça *print* dela). Configure `DEEPSEEK_API_KEY` no seu ambiente antes de rodar a célula que faz a chamada real.

In [ ]:
# Importar o chatbot do projeto (assume que o notebook está na raiz do projeto)
import os
import sys
from pprint import pprint
# garantir que o diretório atual (projeto) esteja no path
sys.path.append('.')
from chatbot_pizzaria import ChatbotPizzaria

# Instanciar (vai treinar o TF-IDF com os patterns de intents.json)
chatbot = ChatbotPizzaria()
print('Chatbot carregado. Intents disponíveis:')
pprint([it['tag'] for it in chatbot.intents.get('intents', [])])

In [ ]:
# Mostrar pratos disponíveis (intent 'escolha_sabor')
pratos = []
for it in chatbot.intents.get('intents', []):
    if it.get('tag') == 'escolha_sabor':
        pratos = list(it.get('patterns', []))
        break
print('Pratos disponíveis:')
pprint(pratos)

## Testes locais (sem chave)
As células abaixo demonstram o comportamento sem necessidade de chave: detecção de intenção e fluxo multi-turn local.

In [ ]:
# Exemplo 1: frase contendo o prato — o chatbot deve detectar e chamar query_deepseek (se chave presente)
texto = 'Quais são os ingredientes da calabresa?'
results = chatbot.process_message(texto)
print('Process message results:')
pprint(results)
# Se a resposta incluir intent 'ingredientes' e o chatbot identificou o prato, você pode chamar query_deepseek diretamente:
dish = chatbot.get_dish_from_text(texto)
print('Prato detectado pela função get_dish_from_text:', dish)

In [ ]:
# Exemplo 2: fluxo multi-turn simulado sem servidor Flask (simula o que o app faz)
msg1 = 'Quais são os ingredientes?'
res1 = chatbot.process_message(msg1)
print('Primeira resposta (esperando prato):')
pprint(res1)
# detectar se o bot está esperando o prato (campo expecting_dish)
waiting = any(r.get('expecting_dish') for r in res1)
print('Bot esperando prato? ', waiting)
# agora simulamos que o usuário responde com '1' (índice)
if waiting:
    user_reply = '1'  # ou 'calabresa'
    dish = chatbot.get_dish_from_text(user_reply)
    print('Prato extraído da resposta do usuário:', dish)
    if dish:
        print('Chamando query_deepseek (se DEEPSEEK_API_KEY estiver definida, fará a chamada real)')
        resp = chatbot.query_deepseek(dish)
        pprint(resp)
    else:
        print('Não foi possível extrair prato da resposta do usuário')

## Teste real com Deepseek/OpenRouter (requer `DEEPSEEK_API_KEY`)
Se você quiser que a célula abaixo faça a chamada real à API, primeiro configure a variável de ambiente `DEEPSEEK_API_KEY` no seu sistema (veja o README). A célula verifica automaticamente se a variável existe e só tentará a requisição quando a chave estiver disponível.

In [ ]:
# Esta célula fará a chamada real apenas se a variável DEEPSEEK_API_KEY estiver definida
if not os.environ.get('DEEPSEEK_API_KEY'):
    print('DEEPSEEK_API_KEY não definida no ambiente. Defina-a antes de rodar esta célula (veja README).')
else:
    # escolha um prato para testar
    test_dish = 'calabresa'
    print('Usando DEEPSEEK_API_KEY presente — consultando Deepseek para', test_dish)
    result = chatbot.query_deepseek(test_dish)
    print('Resposta (truncada):')
    # exibir parte da resposta para evitar poluição na tela
    print(str(result)[:2000])

## Observações finais
- Se a chamada real retornar JSON com estrutura diferente, cole aqui o JSON (sem a chave) e eu ajusto o parsing no código.
- Este notebook é *pronto para chave real*: configure `DEEPSEEK_API_KEY` em PowerShell (ou no ambiente que você usa para rodar o Jupyter) e execute a célula de teste real.
- Se quiser, eu acrescento células que façam mock (pytest) para validar parsing automaticamente.